In [1]:
import keras
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, BatchNormalization, Conv2D, Dropout, Activation, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras import regularizers, optimizers
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

In [4]:
mean = np.mean(x_train, axis=(0, 1, 2, 3))
std = np.std(x_train, axis=(0, 1, 2, 3))
x_train = (x_train - mean)/std
x_test = (x_test - mean)/std
x_vaild = (x_valid - mean)/std

In [5]:
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)
y_valid = to_categorical(y_valid, n_classes)

In [6]:
datagen = ImageDataGenerator(rotation_range=15,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True,
                            vertical_flip=False)
datagen.fit(x_train)

In [7]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3, padding='same', kernel_regularizer=regularizers.l2(1e-4), input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=3, padding='same', kernel_regularizer=regularizers.l2(1e-4), input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, kernel_size=3, padding='same', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=3, padding='same', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=3, padding='same', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=3, padding='same', kernel_regularizer=regularizers.l2(1e-4)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                        

In [8]:
checkpointer = ModelCheckpoint('model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True)

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

In [10]:
model.compile(optimizer=optimizer, 
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [11]:
model.fit(datagen.flow(x_train, y_train, batch_size=128),
                   epochs=125, validation_data=(x_vaild, y_valid), callbacks=[checkpointer],
                   verbose=1)

Epoch 1/125
352/352 [==============================] - 182s 512ms/step - loss: 2.0241 - accuracy: 0.3893 - val_loss: 1.8077 - val_accuracy: 0.4076
Epoch 2/125
352/352 [==============================] - 189s 538ms/step - loss: 1.4672 - accuracy: 0.5225 - val_loss: 1.4274 - val_accuracy: 0.5500
Epoch 3/125
352/352 [==============================] - 207s 587ms/step - loss: 1.2556 - accuracy: 0.5882 - val_loss: 1.0394 - val_accuracy: 0.6668
Epoch 4/125
352/352 [==============================] - 209s 592ms/step - loss: 1.1081 - accuracy: 0.6378 - val_loss: 1.2035 - val_accuracy: 0.6350
Epoch 5/125
352/352 [==============================] - 208s 591ms/step - loss: 1.0131 - accuracy: 0.6694 - val_loss: 0.9944 - val_accuracy: 0.6856
Epoch 6/125
352/352 [==============================] - 200s 568ms/step - loss: 0.9343 - accuracy: 0.6984 - val_loss: 0.7890 - val_accuracy: 0.7464
Epoch 7/125
352/352 [==============================] - 202s 573ms/step - loss: 0.8862 - accuracy: 0.7144 - val_loss: 0

352/352 [==============================] - 190s 539ms/step - loss: 0.5385 - accuracy: 0.8605 - val_loss: 0.5253 - val_accuracy: 0.8698
Epoch 57/125
352/352 [==============================] - 187s 532ms/step - loss: 0.5387 - accuracy: 0.8588 - val_loss: 0.5440 - val_accuracy: 0.8636
Epoch 58/125
352/352 [==============================] - 187s 530ms/step - loss: 0.5363 - accuracy: 0.8609 - val_loss: 0.5607 - val_accuracy: 0.8632
Epoch 59/125
352/352 [==============================] - 182s 516ms/step - loss: 0.5342 - accuracy: 0.8626 - val_loss: 0.5195 - val_accuracy: 0.8756
Epoch 60/125
352/352 [==============================] - 184s 522ms/step - loss: 0.5389 - accuracy: 0.8608 - val_loss: 0.5461 - val_accuracy: 0.8712
Epoch 61/125
352/352 [==============================] - 182s 517ms/step - loss: 0.5325 - accuracy: 0.8634 - val_loss: 0.5143 - val_accuracy: 0.8780
Epoch 62/125
352/352 [==============================] - 182s 516ms/step - loss: 0.5337 - accuracy: 0.8637 - val_loss: 0.5114 

352/352 [==============================] - 183s 520ms/step - loss: 0.5021 - accuracy: 0.8762 - val_loss: 0.4918 - val_accuracy: 0.8824
Epoch 112/125
352/352 [==============================] - 184s 523ms/step - loss: 0.4996 - accuracy: 0.8775 - val_loss: 0.4963 - val_accuracy: 0.8838
Epoch 113/125
352/352 [==============================] - 183s 520ms/step - loss: 0.4978 - accuracy: 0.8764 - val_loss: 0.4759 - val_accuracy: 0.8884
Epoch 114/125
352/352 [==============================] - 184s 524ms/step - loss: 0.4936 - accuracy: 0.8784 - val_loss: 0.5151 - val_accuracy: 0.8772
Epoch 115/125
352/352 [==============================] - 185s 524ms/step - loss: 0.4983 - accuracy: 0.8771 - val_loss: 0.4872 - val_accuracy: 0.8858
Epoch 116/125
352/352 [==============================] - 184s 523ms/step - loss: 0.4997 - accuracy: 0.8773 - val_loss: 0.5359 - val_accuracy: 0.8738
Epoch 117/125
352/352 [==============================] - 183s 519ms/step - loss: 0.4956 - accuracy: 0.8765 - val_loss: 0